In [57]:
from sklearn import model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
import math
%matplotlib inline 
%config InlineBackend.figure_format="retina" 

In [58]:
# 数据处理
continus={}
def ReadData(pathname):
    data=pd.read_csv(pathname)
    data.loc[data['好瓜']=='是', '好瓜'] = 1
    data.loc[data['好瓜']=='否', '好瓜'] = 0
    
    
    data = data.iloc[:,1:]
    
    
    attribute=data.columns
    for i in attribute[:-1]:
        m=0
        if(i!="密度" and i!="含糖率"):
            for j in data[i].unique():
                data.loc[data[i]==j, i]=m
                m+=1
        else:
            for j in data[i].unique():
                continus[i,str(j)]=m
                m+=1
            
    
    label=data['好瓜'].unique()
    
    #print(data)
    #print(continus)
     
    return data,attribute,label

In [59]:
def calContProb(data,feature):
    if(feature=="密度" or feature=="含糖率"):
        return calContProbCon(data,feature)
    else:
        return calContProbDis(data,feature)
        
        
def calContProbCon(data,feature):
    
    prob=np.zeros((len(data['好瓜'].unique()),len(data[feature].unique())))
    
    for i in data['好瓜'].unique():
        for j in data[feature].unique():
            mean=data[data['好瓜']==i][feature].mean()
            std=data[data['好瓜']==i][feature].std()
            
            pos=continus[feature,str(j)]
            prob[i,pos]=exp(-((j-mean)*(j-mean))/(2*std*std))/(sqrt(2*math.pi)*std)
            prob[i,pos]=round(prob[i,pos],3)#保留三位小数
            
            #print(j,prob[i,pos])
            
    print(prob)
    
    return prob
    
def calContProbDis(data,feature):
    
    ni=len(data[feature].unique())
    prob=np.zeros((len(data['好瓜'].unique()),len(data[feature].unique())))
    
    for i in data['好瓜'].unique():
        nc=len(data[data['好瓜']==i])+ni
        for j in data[feature].unique():
            #使用拉普拉斯修正
            prob[i,j]=(len(data[data['好瓜']==i][data[feature]==j])+1)/nc
            
            #不使用拉普拉斯修正
            #prob[i,j]=(len(data[data['好瓜']==i][data[feature]==j]))/(nc-ni)
            
            prob[i,j]=round(prob[i,j],3)#保留三位小数
    
            
    return prob

In [60]:
def trainBayes(data,attribute):
    prob={}
    for i in attribute:
        if i!='好瓜':
            prob[i]=calContProb(pd.DataFrame(data, columns=['好瓜', i]), i)
    return prob

In [61]:
def infer(model,label,attribute,labelData):
    res = {}
    reslog={}
    for i in labels:
        #使用拉普拉斯修正
        res[i] = (len(data[data['好瓜']==i])+1)/(len(data)+2) #N=2
        reslog[i]=log((len(data[data['好瓜']==i])+1)/(len(data)+2))
        
        #不使用拉普拉斯修正
        #res[i] = (len(data[data['好瓜']==i]))/(len(data)) #N=2
        #reslog[i]=log((len(data[data['好瓜']==i]))/(len(data)))
        
        #if i==0:
        #    print(res[i])
        
        for j in attribute:

            res[i]*=model[j][i][0]
            reslog[i] += log(model[j][i][0])
            #print(j,i,model[j][i][0],res[i])

    return res

In [62]:
def judge(res):
    if(res[1]>=res[0]):
        print('判断为好瓜')
    else:
        print('判断为坏瓜')

In [63]:
if __name__ == '__main__':
    data,attribute,labels=ReadData('../../data/watermelon_3.csv')
    model = trainBayes(data,attribute)
    #print(model)
    result = infer(model, labels, attribute[:-1],data['好瓜'])
    judge(result)
    print(result)
    

[[1.203 0.74  1.594 1.737 1.954 1.827 2.043 1.957 1.4   0.88  0.892 1.504
  1.565 1.456 1.605 1.81  1.064]
 [1.959 0.929 2.769 2.981 3.059 1.289 2.386 1.764 2.393 0.117 0.121 0.627
  2.718 2.509 0.786 3.053 1.641]]
[[0.066 0.446 2.203 1.167 3.157 2.756 3.697 3.222 3.116 2.141 2.464 3.246
  3.694 3.408 0.499 2.153 3.306]
 [0.788 2.485 3.911 3.665 3.238 3.629 1.73  3.155 0.701 3.926 0.354 0.809
  2.001 2.87  2.627 0.252 0.868]]
色泽 1 0.364 0.34484210526315784
根蒂 1 0.545 0.18793894736842104
敲声 1 0.636 0.11952917052631579
纹理 1 0.727 0.08689770697263158
脐部 1 0.545 0.04735925030008421
触感 1 0.7 0.03315147521005895
密度 1 1.959 0.06494373993650548
含糖率 1 0.788 0.051175667069966325
色泽 0 0.333 0.3154736842105263
根蒂 0 0.333 0.10505273684210527
敲声 0 0.417 0.043806991263157895
纹理 0 0.25 0.010951747815789474
脐部 0 0.25 0.0027379369539473684
触感 0 0.636 0.0017413279027105263
密度 0 1.203 0.002094817466960763
含糖率 0 0.066 0.00013825795281941037
判断为好瓜
{1: 0.051175667069966325, 0: 0.00013825795281941037}


/Users/wangxu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
